# RFM Analysis

This iPython notebook explains how to perform RFM analysis from customer purchase history data. The sample orders file is Sample - Superstore dataset from Tableau Software.

If you have suggestions or improvements please contribute on https://github.com/joaolcorreia/RFM-analysis

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Read the sample orders file, containing all past purchases for all customers.

In [2]:
orders = pd.read_csv('sample-orders.csv',sep=',')

In [3]:
orders.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franz�sisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


## Create the RFM Table

Since recency is calculated for a point in time and the Tableau Super Store dataset last order date is Dec 31 2014, that is the date we will use to calculate recency.

Set this date to the current day and extract all orders until yesterday.

In [4]:
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [5]:
# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

Create the RFM Table

In [6]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'order_id': lambda x: len(x),      # Frequency
                                        'grand_total': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

## Validating the RFM Table

In [7]:
rfmTable.head()

,frequency,recency,monetary_value
customer,,,
Aaron Bergman,3,415,887
Aaron Hawkins,7,12,1744
Aaron Smayling,7,88,3050
Adam Bellavance,8,54,7756
Adam Hart,10,34,3249


Customer **Aaron Bergman** has frequency:3, monetary value:$887 and recency:415 days.

In [8]:
aaron = orders[orders['customer']=='Aaron Bergman']
aaron

,order_date,order_id,customer,grand_total
624,2011-02-19,CA-2011-152905,Aaron Bergman,13
665,2011-03-07,CA-2011-156587,Aaron Bergman,310
2336,2013-11-11,CA-2013-140935,Aaron Bergman,564


Inserting the date of Aaron purchase and comparing it to the recency in the rfmTable we verify our RFM table is correct.

In [9]:
(NOW - dt.datetime(2013,11,11)).days==415

True

## Determining RFM Quartiles

In [10]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [11]:
quantiles

,frequency,recency,monetary_value
0.25,5.0,30.0,1145.0
0.50,6.0,75.0,2257.0
0.75,8.0,183.0,3784.0


Send quantiles to a dictionary, easier to use.

In [12]:
quantiles = quantiles.to_dict()

In [13]:
quantiles

{'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0},
 'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0}}

## Creating the RFM segmentation table

In [14]:
rfmSegmentation = rfmTable

We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good. 

In [15]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1


In [16]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [17]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [18]:
rfmSegmentation.head()

,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaron Bergman,3,415,887,4,4,4,444
Aaron Hawkins,7,12,1744,1,2,3,123
Aaron Smayling,7,88,3050,3,2,2,322
Adam Bellavance,8,54,7756,2,2,1,221
Adam Hart,10,34,3249,2,1,2,212


In [19]:
# Uncomment any of the following lines to: copy data to clipboard or save it to a CSV file.
# rfmSegmentation.to_clipboard()
# rfmSegmentation.to_csv('rfm-table.csv', sep=',')

Who are the top 5 best customers? by RFM Class (111), high spenders who buy recently and frequently?

In [20]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort('monetary_value', ascending=False).head(5)

,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Sanjit Engle,11,9,12210,1,1,1,111
John Lee,11,21,9801,1,1,1,111
Pete Kriz,12,9,8647,1,1,1,111
Harry Marie,10,2,8237,1,1,1,111
Lena Creighton,12,16,7661,1,1,1,111


In [1]:
import pandas as pd
import numpy as np

# Load the RFM table csv file
rfm_table_path = 'rfm-table.csv'
rfm_table = pd.read_csv(rfm_table_path, encoding='ISO-8859-1')

# Define the product categories
product_categories = ['hoodies', 'shirt', 'tracksuits', 'jeans', 'oversize', 'polo', 
                      'joggers', 'hats', 'trendy sections', 'shoes', 'formals']

# Assign a random product category to each customer
np.random.seed(42)  # For reproducibility
rfm_table['product_interest'] = np.random.choice(product_categories, size=len(rfm_table))

# Define promotion and update frequency based on RFM scores
def set_promotion_update_frequency(row):
    if row['RFMClass'] == 444:
        return 'Once a month', 'No updates'
    elif row['R_Quartile'] == 1 or row['M_Quartile'] == 1:
        return 'Every 2 days', 'Every update'
    elif row['F_Quartile'] == 1:
        return 'Every week', 'Every update'
    else:
        return 'Every 2 weeks', 'Updates on interest'

# Apply the function to each row
rfm_table[['promotion_frequency', 'update_frequency']] = rfm_table.apply(set_promotion_update_frequency, axis=1, result_type="expand")

# Save the updated DataFrame to a new CSV file
updated_rfm_table_path = 'updated_rfm_table.csv'
rfm_table.to_csv(updated_rfm_table_path, index=False)

# Display the first few rows of the updated RFM table
rfm_table.head()


,customer,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,product_interest,promotion_frequency,update_frequency
0,Aaron Bergman,3,415,887,4,4,4,444,joggers,Once a month,No updates
1,Aaron Hawkins,7,12,1744,1,2,3,123,jeans,Every 2 days,Every update
2,Aaron Smayling,7,88,3050,3,2,2,322,formals,Every 2 weeks,Updates on interest
3,Adam Bellavance,8,54,7756,2,2,1,221,hats,Every 2 days,Every update
4,Adam Hart,10,34,3249,2,1,2,212,oversize,Every week,Every update


In [3]:
import pandas as pd
import numpy as np

# Load the RFM table csv file
rfm_table_path = 'rfm-table.csv'
rfm_table = pd.read_csv(rfm_table_path, encoding='ISO-8859-1')

# Define the product categories
product_categories = ['hoodies', 'shirt', 'tracksuits', 'jeans', 'oversize', 'polo', 
                      'joggers', 'hats', 'trendy sections', 'shoes', 'formals']

# Assign a random product category to each customer
np.random.seed(42)  # For reproducibility
rfm_table['product_interest'] = np.random.choice(product_categories, size=len(rfm_table))

# Define the promotion frequency options
promotion_frequency_options = [
    'Once everyday', 'Once every 2 days', 'Once every 3 days',
    'Once every week', 'Once every 2 weeks', 'Once every month', 'Once every 2 months'
]

# Define the update frequency options
update_frequency_options = [
    'Every update on interested category',
    'Updates on interests',
    'Updates on interested products alone',
    'Updates on viewed products only',
    'No updates'
]

# Define promotion and update frequency based on RFM scores
def set_promotion_update_frequency(row):
    if row['RFMClass'] == 444:
        promotion_frequency = promotion_frequency_options[6]  # Once every 2 months
        update_frequency = update_frequency_options[4]  # No updates
    elif row['R_Quartile'] == 1 or row['M_Quartile'] == 1:
        promotion_frequency = promotion_frequency_options[0]  # Once everyday
        update_frequency = update_frequency_options[0]  # Every update on interested category
    elif row['F_Quartile'] == 1:
        promotion_frequency = promotion_frequency_options[3]  # Once every week
        update_frequency = update_frequency_options[0]  # Every update on interested category
    else:
        promotion_frequency = promotion_frequency_options[4]  # Once every 2 weeks
        update_frequency = update_frequency_options[1]  # Updates on interests
    return promotion_frequency, update_frequency

# Apply the function to each row
rfm_table[['promotion_frequency', 'update_frequency']] = rfm_table.apply(set_promotion_update_frequency, axis=1, result_type="expand")

# Save the updated DataFrame to a new CSV file
updated_rfm_table_path = 'updated_rfm_table.csv'
rfm_table.to_csv(updated_rfm_table_path, index=False)

# Display the first few rows of the updated RFM table
rfm_table.head()


,customer,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,product_interest,promotion_frequency,update_frequency
0,Aaron Bergman,3,415,887,4,4,4,444,joggers,Once every 2 months,No updates
1,Aaron Hawkins,7,12,1744,1,2,3,123,jeans,Once everyday,Every update on interested category
2,Aaron Smayling,7,88,3050,3,2,2,322,formals,Once every 2 weeks,Updates on interests
3,Adam Bellavance,8,54,7756,2,2,1,221,hats,Once everyday,Every update on interested category
4,Adam Hart,10,34,3249,2,1,2,212,oversize,Once every week,Every update on interested category


In [5]:
import pandas as pd
import numpy as np

# Load the RFM table csv file
rfm_table_path = 'rfm-table.csv'
rfm_table = pd.read_csv(rfm_table_path, encoding='ISO-8859-1')

# Define the product categories
product_categories = ['hoodies', 'shirt', 'tracksuits', 'jeans', 'oversize', 'polo', 
                      'joggers', 'hats', 'trendy sections', 'shoes', 'formals']

# Assign a random product category to each customer
np.random.seed(42)  # For reproducibility
rfm_table['product_interest'] = np.random.choice(product_categories, size=len(rfm_table))

# Define the promotion frequency options
promotion_frequency_options = [
    'Once everyday', 'Once every 2 days', 'Once every 3 days',
    'Once every week', 'Once every 2 weeks', 'Once every month', 'Once every 2 months'
]

# Define the update frequency options
update_frequency_mapping = {
    1: 'Every update on interested category',
    2: 'Updates on interests',
    3: 'Updates on interested products alone',
    4: 'Updates on viewed products only',
    5: 'No updates'
}

# Define promotion and update frequency based on RFM scores
def set_promotion_update_frequency(row):
    if row['RFMClass'] == 444:
        promotion_frequency = promotion_frequency_options[6]  # Once every 2 months
        update_frequency = 5  # No updates
    elif row['R_Quartile'] == 1 or row['M_Quartile'] == 1:
        promotion_frequency = promotion_frequency_options[0]  # Once everyday
        update_frequency = 1  # Every update on interested category
    elif row['F_Quartile'] == 1:
        promotion_frequency = promotion_frequency_options[3]  # Once every week
        update_frequency = 1  # Every update on interested category
    else:
        promotion_frequency = promotion_frequency_options[4]  # Once every 2 weeks
        update_frequency = 2  # Updates on interests
    return promotion_frequency, update_frequency

# Apply the function to each row
rfm_table[['promotion_frequency', 'update_frequency']] = rfm_table.apply(set_promotion_update_frequency, axis=1, result_type="expand")

# Save the updated DataFrame to a new CSV file
updated_rfm_table_path = 'updated_rfm_table.csv'
rfm_table.to_csv(updated_rfm_table_path, index=False)

# Display the first few rows of the updated RFM table
rfm_table.head()

# Print the first few rows of the update frequency mapping for verification
for key, value in update_frequency_mapping.items():
    print(f"{key}: {value}")


1: Every update on interested category
2: Updates on interests
3: Updates on interested products alone
4: Updates on viewed products only
5: No updates


In [9]:
import pandas as pd
import numpy as np

# Load the RFM table csv file
rfm_table_path = 'rfm-table.csv'
rfm_table = pd.read_csv(rfm_table_path, encoding='ISO-8859-1')

# Define the product categories
product_categories = ['Hoodies', 'Shirt', 'Tracksuits', 'Jeans', 'Oversize', 'Polo', 
                      'Joggers', 'Hats', 'Trendy Sections', 'Shoes', 'Formals']

# Assign a random product category to each customer
np.random.seed(42)  # For reproducibility
rfm_table['product_interest'] = np.random.choice(product_categories, size=len(rfm_table))

# Define the promotion frequency options
promotion_frequency_options = [
    'Once everyday', 'Once every 2 days', 'Once every 3 days',
    'Once every week', 'Once every 2 weeks', 'Once every month', 'Once every 2 months'
]

# Define the update frequency ranking
def rank_update_frequency(row):
    # Generate a rank based on RFM class
    # Higher RFM scores (worst) get higher update frequency numbers (less frequent updates)
    if row['RFMClass'] == 444:
        return 5  # Worst RFM rank
    elif row['R_Quartile'] == 1 or row['M_Quartile'] == 1:
        return 1  # Best RFM rank
    elif row['F_Quartile'] == 1:
        return 2
    else:
        return 3

# Apply the ranking function to each row
rfm_table['update_frequency'] = rfm_table.apply(rank_update_frequency, axis=1)

# Define promotion frequency based on RFM scores
def set_promotion_frequency(row):
    if row['RFMClass'] == 444:
        return promotion_frequency_options[6]  # Once every 2 months
    elif row['R_Quartile'] == 1 or row['M_Quartile'] == 1:
        return promotion_frequency_options[0]  # Once everyday
    elif row['F_Quartile'] == 1:
        return promotion_frequency_options[3]  # Once every week
    else:
        return promotion_frequency_options[4]  # Once every 2 weeks

# Apply the function to each row
rfm_table['promotion_frequency'] = rfm_table.apply(set_promotion_frequency, axis=1)

# Save the updated DataFrame to a new CSV file
updated_rfm_table_path = 'updated_rfm_table.csv'
rfm_table.to_csv(updated_rfm_table_path, index=False)

# Display the first few rows of the updated RFM table
rfm_table.head()


,customer,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,product_interest,update_frequency,promotion_frequency
0,Aaron Bergman,3,415,887,4,4,4,444,Joggers,5,Once every 2 months
1,Aaron Hawkins,7,12,1744,1,2,3,123,Jeans,1,Once everyday
2,Aaron Smayling,7,88,3050,3,2,2,322,Formals,3,Once every 2 weeks
3,Adam Bellavance,8,54,7756,2,2,1,221,Hats,1,Once everyday
4,Adam Hart,10,34,3249,2,1,2,212,Oversize,2,Once every week


In [11]:
import pandas as pd
import numpy as np

# Load the RFM table csv file
rfm_table_path = 'rfm-table.csv'
rfm_table = pd.read_csv(rfm_table_path, encoding='ISO-8859-1')

# Define the product categories
product_categories = ['Hoodies', 'Shirt', 'Tracksuits', 'Jeans', 'Oversize', 'Polo', 
                      'Joggers', 'Hats', 'Trendy Sections', 'Shoes', 'Formals']

# Assign one or two random product categories to each customer
np.random.seed(42)  # For reproducibility
def assign_product_interests():
    num_interests = np.random.choice([1, 2])
    return ', '.join(np.random.choice(product_categories, num_interests, replace=False))

rfm_table['product_interest'] = [assign_product_interests() for _ in range(len(rfm_table))]

# Define the promotion frequency options
promotion_frequency_options = [
    'Once everyday', 'Once every 2 days', 'Once every 3 days',
    'Once every week', 'Once every 2 weeks', 'Once every month', 'Once every 2 months'
]

# Define the update frequency ranking
def rank_update_frequency(row):
    # Generate a rank based on RFM class
    # Higher RFM scores (worst) get higher update frequency numbers (less frequent updates)
    if row['RFMClass'] == 444:
        return 5  # Worst RFM rank
    elif row['R_Quartile'] == 1 or row['M_Quartile'] == 1:
        return 1  # Best RFM rank
    elif row['F_Quartile'] == 1:
        return 2
    else:
        return 3

# Apply the ranking function to each row
rfm_table['update_frequency'] = rfm_table.apply(rank_update_frequency, axis=1)

# Define promotion frequency based on RFM scores
def set_promotion_frequency(row):
    if row['RFMClass'] == 444:
        return promotion_frequency_options[6]  # Once every 2 months
    elif row['R_Quartile'] == 1 or row['M_Quartile'] == 1:
        return promotion_frequency_options[0]  # Once everyday
    elif row['F_Quartile'] == 1:
        return promotion_frequency_options[3]  # Once every week
    else:
        return promotion_frequency_options[4]  # Once every 2 weeks

# Apply the function to each row
rfm_table['promotion_frequency'] = rfm_table.apply(set_promotion_frequency, axis=1)

# Save the updated DataFrame to a new CSV file
updated_rfm_table_path = 'updated_rfm_table.csv'
rfm_table.to_csv(updated_rfm_table_path, index=False)

# Display the first few rows of the updated RFM table
rfm_table.head()


,customer,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,product_interest,update_frequency,promotion_frequency
0,Aaron Bergman,3,415,887,4,4,4,444,Formals,5,Once every 2 months
1,Aaron Hawkins,7,12,1744,1,2,3,123,"Formals, Shoes",1,Once everyday
2,Aaron Smayling,7,88,3050,3,2,2,322,"Hats, Oversize",3,Once every 2 weeks
3,Adam Bellavance,8,54,7756,2,2,1,221,"Shirt, Shoes",1,Once everyday
4,Adam Hart,10,34,3249,2,1,2,212,Tracksuits,2,Once every week


In [11]:
import pandas as pd

# Load the updated RFM table csv file
updated_rfm_table_path = 'updated_rfm_table.csv'
rfm_table = pd.read_csv(updated_rfm_table_path, encoding='ISO-8859-1')

# Function to search for a customer's records by name
def search_customer_records(customer_name):
    # Search for records matching the customer's name
    customer_records = rfm_table[rfm_table['customer'].str.contains(customer_name, case=False, na=False)]
    return customer_records

# Get user input for the customer's name
customer_name = input("Enter the customer's name to search: ")

# Search for and display the customer's records
customer_records = search_customer_records(customer_name)
if not customer_records.empty:
    print("Customer records found:")
    print(customer_records)
else:
    print("No records found for the customer name provided.")



Enter the customer's name to search:  Adam Hart


Customer records found:
    customer  frequency  recency  monetary_value  R_Quartile  F_Quartile  \
4  Adam Hart         10       34            3249           2           1   

   M_Quartile  RFMClass product_interest  update_frequency promotion_frequency  
4           2       212       Tracksuits                 2     Once every week  
